<a href="https://colab.research.google.com/github/Paolino1994/IntroIA/blob/main/Integrador%20Clase%207/Implementaci%C3%B3n_GMM_Paolino_Tourn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import numpy as np
from scipy.stats import multivariate_normal
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [169]:
class EMScalar():
    
    def fit(self,X,qDist=2,qIter=15): #parametros):
        # Los parámetros deben incluir al menos:
        #  - Alguna forma de detener la iteración
        #  - Los datos observados
        #  - La cantidad de distribuciones 
        
        # Inicialización de parámetros        
          # Inicializar las probabilidades marginales de las clases P(z)
        self.pi=np.random.rand(qDist)
        self.pi=self.pi/qDist
          # Inicializar medias
        self.mu=X.mean(axis=1)
          # Inicializar matrices covarianza
        #self.sigma=np.cov(X,bias=True)
        self.sigma = np.zeros((len(X),qDist))
        # Crear matrices place-holders para 
            # p(x|z) para cada clase z (n x k), [p(x1|z1) p(x1|z2) p(x1|z3) ..]
            # Responsibilities
        pxZ=np.ones((len(X),qDist))
        # Calcular, con los parámetros iniciales, p(x|z) para todos los z
        #Si no anda, usar con un for
        for i in range(qDist):
          pxZ[i]= multivariate_normal.pdf(X, mean=self.mu[i], cov=self.sigma[i])
        
        # Algoritmo de actualización 
        counter=0
        reponsibilities=np.zeros((len(X),qDist))        
        while counter<qIter:          
          for j in range(qDist):          
                # Responsibilities
            reponsibilitieJ=self.pi[j]*pxZ[:,j]/np.sum(self.pi*pxZ,axis=1)
            reponsibilities[:,j]=reponsibilitieJ
            # Calcular Nk, este es de prueba
            Nk=np.sum(reponsibilitieJ)
                # Actualizar medias 
            self.mu[j]=np.sum(reponsibilitieJ*X[j])/Nk
                # Actualizar covarianzas
            self.sigma[j]=reponsibilitieJ*(X[:,j]-self.mu[j])*(X[:,j]-self.mu[j])/Nk
                # Actualizar pesos de clases
            self.pi[j]=np.mean(reponsibilitieJ)
                # Actualizar p(x|z)
            pxZ[j]= multivariate_normal.pdf(X, mean=self.mu[j], cov=self.sigma[j])

            counter+=1

    def predict(self,X): #inputs necesarios):
        # Devuelve para cada observación la clase asignada
        q = self.mu.shape[0]
        N = np.zeros((X.shape[0], q))
        E = np.zeros((X.shape[0], q))
        
        for i in range(q):
            N[:, i] = multivariate_normal.pdf(X, self.mu[i, 0], self.sigma[i, 0])
        for i in range(q):
            E[:, i] = (self.pi[i, 0] * N[:, i]) / (N @ self.pi)[:, 0]
        idx = np.argmax(E, axis=1)
        return idx